In [1]:
# grabbed from https://github.com/L1aoXingyu/pytorch-beginner/blob/master/05-Recurrent%20Neural%20Network/recurrent_network.py

In [2]:
__author__ = 'SherlockLiao'

import torch
from torch import nn, optim
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision import datasets

# 定义超参数
batch_size = 100
learning_rate = 1e-3
num_epoches = 20

# 下载训练集 MNIST 手写数字训练集
train_dataset = datasets.MNIST(
    root='./data', train=True, transform=transforms.ToTensor(), download=True)

test_dataset = datasets.MNIST(
    root='./data', train=False, transform=transforms.ToTensor())

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [3]:
# 定义 Recurrent Network 模型
class Rnn(nn.Module):
    def __init__(self, in_dim, hidden_dim, n_layer, n_class):
        super(Rnn, self).__init__()
        self.n_layer = n_layer
        self.hidden_dim = hidden_dim
        self.lstm = nn.LSTM(in_dim, hidden_dim, n_layer, batch_first=True)
        self.classifier = nn.Linear(hidden_dim, n_class)

    def forward(self, x):
        # h0 = Variable(torch.zeros(self.n_layer, x.size(1),
        #   self.hidden_dim)).cuda()
        # c0 = Variable(torch.zeros(self.n_layer, x.size(1),
        #   self.hidden_dim)).cuda()
        out, _ = self.lstm(x)
        out = out[:, -1, :]
        out = self.classifier(out)
        return out

In [4]:
model = Rnn(28, 128, 2, 10)  # 图片大小是28x28
use_gpu = torch.cuda.is_available()  # 判断是否有GPU加速
if use_gpu:
    model = model.cuda()
# 定义loss和optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [5]:
# 开始训练
for epoch in range(num_epoches):
    print('epoch {}'.format(epoch + 1))
    print('*' * 10)
    running_loss = 0.0
    running_acc = 0.0
    for i, data in enumerate(train_loader, 1):
        img, label = data
        b, c, h, w = img.size()
        assert c == 1, 'channel must be 1'
        img = img.squeeze(1)
        # img = img.view(b*h, w)
        # img = torch.transpose(img, 1, 0)
        # img = img.contiguous().view(w, b, -1)
        if use_gpu:
            img = Variable(img).cuda()
            label = Variable(label).cuda()
        else:
            img = Variable(img)
            label = Variable(label)
        # 向前传播
        out = model(img)
        loss = criterion(out, label)
        running_loss += loss.item() * label.size(0)
        _, pred = torch.max(out, 1)
        num_correct = (pred == label).sum()
        running_acc += num_correct.item()
        # 向后传播
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if i % 300 == 0:
            print('[{}/{}] Loss: {:.6f}, Acc: {:.6f}'.format(
                epoch + 1, num_epoches, running_loss / (batch_size * i),
                running_acc / (batch_size * i)))
    print('Finish {} epoch, Loss: {:.6f}, Acc: {:.6f}'.format(
        epoch + 1, running_loss / (len(train_dataset)), running_acc / (len(
            train_dataset))))
    model.eval()
    eval_loss = 0.
    eval_acc = 0.
    with torch.no_grad():
        for data in test_loader:
            img, label = data
            b, c, h, w = img.size()
            assert c == 1, 'channel must be 1'
            img = img.squeeze(1)

            if use_gpu:
                img = Variable(img).cuda()
                label = Variable(label).cuda()
            else:
                img = Variable(img)
                label = Variable(label)
            out = model(img)
            loss = criterion(out, label)
            eval_loss += loss.item() * label.size(0)
            _, pred = torch.max(out, 1)
            num_correct = (pred == label).sum()
            eval_acc += num_correct.item()
        print('Test Loss: {:.6f}, Acc: {:.6f}'.format(eval_loss / (len(
            test_dataset)), eval_acc / (len(test_dataset))))
    print()

# 保存模型
torch.save(model.state_dict(), './rnn.pth')

epoch 1
**********
[1/20] Loss: 0.778067, Acc: 0.734300
[1/20] Loss: 0.495055, Acc: 0.835600
Finish 1 epoch, Loss: 0.495055, Acc: 0.835600
Test Loss: 0.183299, Acc: 0.939500

epoch 2
**********
[2/20] Loss: 0.136879, Acc: 0.957700
[2/20] Loss: 0.121418, Acc: 0.962983
Finish 2 epoch, Loss: 0.121418, Acc: 0.962983
Test Loss: 0.098300, Acc: 0.971300

epoch 3
**********
[3/20] Loss: 0.077217, Acc: 0.975800
[3/20] Loss: 0.078388, Acc: 0.976017
Finish 3 epoch, Loss: 0.078388, Acc: 0.976017
Test Loss: 0.064563, Acc: 0.979200

epoch 4
**********
[4/20] Loss: 0.057657, Acc: 0.982167
[4/20] Loss: 0.056992, Acc: 0.982350
Finish 4 epoch, Loss: 0.056992, Acc: 0.982350
Test Loss: 0.075820, Acc: 0.976600

epoch 5
**********
[5/20] Loss: 0.043867, Acc: 0.986267
[5/20] Loss: 0.046119, Acc: 0.985633
Finish 5 epoch, Loss: 0.046119, Acc: 0.985633
Test Loss: 0.046326, Acc: 0.986500

epoch 6
**********
[6/20] Loss: 0.040464, Acc: 0.988033
[6/20] Loss: 0.040219, Acc: 0.988100
Finish 6 epoch, Loss: 0.040219, 